## Term Project Milestone 4
### Course: DSC 540
### Author: Holly Figueroa
### Objective: Transformations of API data source (CMS.gov Nursing Home Data)

In [1]:
import pandas as pd
import json
import requests
from pandas import DataFrame
import seaborn as sns

In [2]:
# # Create url variable for webpage
# url_1 = 'https://data.cms.gov/provider-data/api/1/datastore/query/f210d227-2b26-5802-9f3b-e65d11aa9354?keys=true&limit=1&offset=0'

# # Request data from the url
# response = requests.get(url_1)

In [3]:
# # Load response as json
# text = json.loads(response.text)

In [4]:
# # API page shows headers for data nested within a library, under 'query'. View keys of initial json library
# for key in text.keys():
#     print(key)

In [5]:
# print(text['query'])

In [6]:
# Headers are kept as a list under key "properties"
# headers = text['query']['properties']

In [7]:
# # View headers - use to create empty dataframe
# x = 0
# for header in headers:
#     print(x, header)
#     x +=1

In [8]:
# # Json requests limited to 500 rows, Table contains 15203 rows - calculate number of loops needed to get all rows
# 15203/500

Web page states table contains 15203 rows of data. Json page limits query results to 500. Will have to loop through 31 times to obtain all 15k + rows of data. Page being used is here(https://data.cms.gov/provider-data/api/1/datastore/query/f210d227-2b26-5802-9f3b-e65d11aa9354?keys=true&limit=500&offset=0)

In [9]:
# # Create a loop to gather rows of data from API and append them into dataframe.
# # Set offset paramter to 0 which will be increased each loop take to next page of data
# offset = 0

# # Create dataframe of headers only to add data
# nursing_df = pd.DataFrame(columns = headers)

# # Loop over 500 rows of data 31 times,to get all rows of data
# for x in range (0,31):
#     url = f'https://data.cms.gov/provider-data/api/1/datastore/query/f210d227-2b26-5802-9f3b-e65d11aa9354?keys=true&limit=500&offset={offset}'
#     response = requests.get(url)
#     text = json.loads(response.text)
    
#     # Create dataframe from rows contained in key 'results'
#     rows = text['results']
#     new_df = DataFrame(rows)
    
#     # concat df of rows to build dataframe
#     nursing_df = pd.concat([nursing_df, new_df], axis = 0)
    
#     # Increase offset parameter by 500 so url will bring a new next page of rows
#     offset += 500

In [10]:
nursing_df = pd.read_csv('NH_ProviderInfo_May2022.csv')

In [11]:
# Check shape matches API description of data as having 15203 rows
nursing_df.shape

(15197, 96)

1. Drop Columns that will not be used - Dropping individual health survey scores and keeping the total weighted score, dropping footnote columns, and others that do not appear as helpful as others. For values that have multiple measures, I will opt for the adjusted measures.


In [12]:
# View columns and index
for col in nursing_df.columns:
    index_no = nursing_df.columns.get_loc(col)
    print(index_no , col, )

0 Federal Provider Number
1 Provider Name
2 Provider Address
3 Provider City
4 Provider State
5 Provider Zip Code
6 Provider Phone Number
7 Provider SSA County Code
8 Provider County Name
9 Ownership Type
10 Number of Certified Beds
11 Average Number of Residents per Day
12 Average Number of Residents per Day Footnote
13 Provider Type
14 Provider Resides in Hospital
15 Legal Business Name
16 Date First Approved to Provide Medicare and Medicaid Services
17 Continuing Care Retirement Community
18 Special Focus Status
19 Abuse Icon
20 Most Recent Health Inspection More Than 2 Years Ago
21 Provider Changed Ownership in Last 12 Months
22 With a Resident and Family Council
23 Automatic Sprinkler Systems in All Required Areas
24 Overall Rating
25 Overall Rating Footnote
26 Health Inspection Rating
27 Health Inspection Rating Footnote
28 QM Rating
29 QM Rating Footnote
30 Long-Stay QM Rating
31 Long-Stay QM Rating Footnote
32 Short-Stay QM Rating
33 Short-Stay QM Rating Footnote
34 Staffing Ra

In [13]:
# Transformation [1]
# Drop uneeded columns by index
nursing_drop = nursing_df.drop(
    nursing_df.columns[[2,3,5,6,12,14,16,17,18,19,20,21,22,23,25,27,29,31,33,35,37,38,39,40,41,
                        42,43,44,45,46,47,49,51,53,54,55,56,57,62,63,64,65,66,67,68,69,70,
                        71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,94,95],], axis = 1)

In [14]:
# Print columns in list format to copy/paste/edit and replace.
nursing_drop.columns

Index(['Federal Provider Number', 'Provider Name', 'Provider State',
       'Provider SSA County Code', 'Provider County Name', 'Ownership Type',
       'Number of Certified Beds', 'Average Number of Residents per Day',
       'Provider Type', 'Legal Business Name', 'Overall Rating',
       'Health Inspection Rating', 'QM Rating', 'Long-Stay QM Rating',
       'Short-Stay QM Rating', 'Staffing Rating', 'RN Staffing Rating',
       'Total nursing staff turnover', 'Registered Nurse turnover',
       'Number of administrators who have left the nursing home',
       'Adjusted Nurse Aide Staffing Hours per Resident per Day',
       'Adjusted LPN Staffing Hours per Resident per Day',
       'Adjusted RN Staffing Hours per Resident per Day',
       'Adjusted Total Nurse Staffing Hours per Resident per Day',
       'Total Weighted Health Survey Score',
       'Number of Facility Reported Incidents',
       'Number of Substantiated Complaints',
       'Number of Citations from Infection Control

2. Next I will replace headers to simplify.

In [15]:
# Transformation [2]
# Replace headers (34 total) to simplify 
# Create list of new column names (some old) for each of the 20 columns
new_headers = [
    'provider_number',
    'provider_name',
    'state',
    'ssa_county_code', 
    'county', 
    'ownership_type',
    'number_beds', 
    'average_residents',
    'provider_type', 
    'legal_name',
    'overall_rating',
    'health_insp_rating', 
    'qm_rating', 
    'longstay_qm_rating',
    'shortstay_qm_rating', 
    'staffing_rating', 
    'rn_staffing_rating',
    'nursing_turnover', 
    'rn_turnover',
    'admin_turnover',
    'nurse_aide_hrspres',
    'lpn_hrspres',
    'rn_hrspres',
    'total_nurse_hrspres',
    'total_weighted_health_survey_score',
    'reported_incidents',
    'substantiated_complaints',
    'infection_control_citations',
    'num_fines', 
    'fines_dollar_total',
    'num_payment_denials', 
    'total_penalties',
        ]

In [16]:
# Review columns side by side before replacing
for header1,header2 in zip(nursing_drop.columns, new_headers):
    print(header1)
    print(header2)
    print('\n')
    

Federal Provider Number
provider_number


Provider Name
provider_name


Provider State
state


Provider SSA County Code
ssa_county_code


Provider County Name
county


Ownership Type
ownership_type


Number of Certified Beds
number_beds


Average Number of Residents per Day
average_residents


Provider Type
provider_type


Legal Business Name
legal_name


Overall Rating
overall_rating


Health Inspection Rating
health_insp_rating


QM Rating
qm_rating


Long-Stay QM Rating
longstay_qm_rating


Short-Stay QM Rating
shortstay_qm_rating


Staffing Rating
staffing_rating


RN Staffing Rating
rn_staffing_rating


Total nursing staff turnover
nursing_turnover


Registered Nurse turnover
rn_turnover


Number of administrators who have left the nursing home
admin_turnover


Adjusted Nurse Aide Staffing Hours per Resident per Day
nurse_aide_hrspres


Adjusted LPN Staffing Hours per Resident per Day
lpn_hrspres


Adjusted RN Staffing Hours per Resident per Day
rn_hrspres


Adjusted Total Nurse S

In [17]:
# Replace headers
nursing_drop.columns = new_headers

In [18]:
# Check for null values
# Looping through columns
for col in nursing_drop:
    # print whether any values in the column are null
    print(any(nursing_drop[col].isnull()), "-----", col)

False ----- provider_number
False ----- provider_name
False ----- state
False ----- ssa_county_code
False ----- county
False ----- ownership_type
False ----- number_beds
True ----- average_residents
False ----- provider_type
False ----- legal_name
True ----- overall_rating
True ----- health_insp_rating
True ----- qm_rating
True ----- longstay_qm_rating
True ----- shortstay_qm_rating
True ----- staffing_rating
True ----- rn_staffing_rating
True ----- nursing_turnover
True ----- rn_turnover
True ----- admin_turnover
True ----- nurse_aide_hrspres
True ----- lpn_hrspres
True ----- rn_hrspres
True ----- total_nurse_hrspres
True ----- total_weighted_health_survey_score
False ----- reported_incidents
False ----- substantiated_complaints
True ----- infection_control_citations
False ----- num_fines
False ----- fines_dollar_total
False ----- num_payment_denials
False ----- total_penalties


In [19]:
# View df head
nursing_drop.head()

,provider_number,provider_name,state,ssa_county_code,county,ownership_type,number_beds,average_residents,provider_type,legal_name,...,rn_hrspres,total_nurse_hrspres,total_weighted_health_survey_score,reported_incidents,substantiated_complaints,infection_control_citations,num_fines,fines_dollar_total,num_payment_denials,total_penalties
0,015009,"BURNS NURSING HOME, INC.",AL,290,Franklin,For profit - Corporation,57,51.1,Medicare and Medicaid,"BURNS NURSING HOME, INC.",...,1.28139,4.18288,5.333,0,0,0.0,1,655.14,0,1
1,015010,COOSA VALLEY HEALTHCARE CENTER,AL,600,Talladega,For profit - Corporation,85,77.6,Medicare and Medicaid,COOSA VALLEY HEALTHCARE CENTER LLC,...,1.09301,4.52666,6.667,0,0,0.0,1,650.00,0,1
2,015012,HIGHLANDS HEALTH AND REHAB,AL,350,Jackson,Government - County,50,35.8,Medicare and Medicaid,JACKSON COUNTY HEALTH CARE AUTHORITY,...,NaN,NaN,13.333,0,0,0.0,0,0.00,0,0
3,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,AL,360,Jefferson,For profit - Individual,92,71.4,Medicare and Medicaid,BALL HEALTHCARE EASTVIEW INC,...,0.67048,3.72369,12.667,0,0,0.0,0,0.00,0,0
4,015015,PLANTATION MANOR NURSING HOME,AL,360,Jefferson,For profit - Individual,103,73.8,Medicare and Medicaid,"C & G HEALTHCARE SERVICES, INC.",...,0.47964,4.07314,16.000,0,1,0.0,3,8333.00,0,3


3. I will look at variable data types to make them compatible for calculations and string methods if needed

In [20]:
# Check data types first so they are compatible for plots and calculations
nursing_drop.dtypes

provider_number                        object
provider_name                          object
state                                  object
ssa_county_code                         int64
county                                 object
ownership_type                         object
number_beds                             int64
average_residents                     float64
provider_type                          object
legal_name                             object
overall_rating                        float64
health_insp_rating                    float64
qm_rating                             float64
longstay_qm_rating                    float64
shortstay_qm_rating                   float64
staffing_rating                       float64
rn_staffing_rating                    float64
nursing_turnover                      float64
rn_turnover                           float64
admin_turnover                        float64
nurse_aide_hrspres                    float64
lpn_hrspres                       

In [21]:
# First I must change data types of columns to numeric first batch
# create variables of the column ranges I want to change
cols1 = nursing_drop.columns[10:]
cols2 = nursing_drop.columns[6:8]
print(cols1, cols2)

Index(['overall_rating', 'health_insp_rating', 'qm_rating',
       'longstay_qm_rating', 'shortstay_qm_rating', 'staffing_rating',
       'rn_staffing_rating', 'nursing_turnover', 'rn_turnover',
       'admin_turnover', 'nurse_aide_hrspres', 'lpn_hrspres', 'rn_hrspres',
       'total_nurse_hrspres', 'total_weighted_health_survey_score',
       'reported_incidents', 'substantiated_complaints',
       'infection_control_citations', 'num_fines', 'fines_dollar_total',
       'num_payment_denials', 'total_penalties'],
      dtype='object') Index(['number_beds', 'average_residents'], dtype='object')


In [22]:
# Transformation[3]
# Apply conversion 'to numeric' accross columns I have chosen
nursing_drop[cols1] = nursing_drop[cols1].apply(pd.to_numeric, errors='raise')
nursing_drop[cols2] = nursing_drop[cols2].apply(pd.to_numeric, errors='raise')

4. Build a County dataframe - create county level variables to include. This will require subsetting, grouping, merging, and new variable creations. I will start with a small subset of data, and merge additional dataframes at the county level with it. Some variables will be aggregated, others will be summed at the county level before merging.

In [23]:
# Create initial df of columns for a county df from the nursing homes provider df
county_build = nursing_drop[['state','county']]

In [24]:
county_build['state_county'] = (county_build['state'] + ', ' + county_build['county'])

<ipython-input-24-2f1b72c1c771>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_build['state_county'] = (county_build['state'] + ', ' + county_build['county'])


In [25]:
county_build.head()

,state,county,state_county
0,AL,Franklin,"AL, Franklin"
1,AL,Talladega,"AL, Talladega"
2,AL,Jackson,"AL, Jackson"
3,AL,Jefferson,"AL, Jefferson"
4,AL,Jefferson,"AL, Jefferson"


In [26]:
# Create a df of value counts for each couty using state_county  - resetting the index to be named later
county_counts = pd.DataFrame(county_build['state_county'].value_counts().reset_index())

In [27]:
# give new count df
county_counts.columns = ['state_county', 'nursing_home_count']

# View count df
county_counts.head()

,state_county,nursing_home_count
0,"CA, Los Angeles",379
1,"IL, Cook",206
2,"TX, Harris",102
3,"OH, Cuyahoga",97
4,"TX, Dallas",87


In [28]:
# Transformation [4] drop duplicates
# Get a county-level df from my county build df by dropping duplicates of county code, keeping the first occurance only
county = county_build.drop_duplicates(subset = ['state_county'], keep = 'first')

In [29]:
#county = county.astype(str)

In [30]:
# Transformation [5]
# Merge counts with counties
county = county.merge(county_counts, on=['state_county'], how= 'inner')
#county.drop([''])
# View df
county.head()

,state,county,state_county,nursing_home_count
0,AL,Franklin,"AL, Franklin",4
1,AL,Talladega,"AL, Talladega",3
2,AL,Jackson,"AL, Jackson",3
3,AL,Jefferson,"AL, Jefferson",34
4,AL,Limestone,"AL, Limestone",2


In [31]:
# Transformation [6] groupby.sum()
# Gather other variables form the nursing df to group by and merge at county levels
# First I will gather variables to group by using a sum function
sum_variables = nursing_drop.iloc[:,[2,4,25,26,27,28,29,30,31]]
sum_variables['state_county'] = sum_variables['state'] + ', '+ sum_variables['county']

<ipython-input-31-49e3102e4d72>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_variables['state_county'] = sum_variables['state'] + ', '+ sum_variables['county']


In [32]:
sum_variables = sum_variables.groupby(by = ['state_county']).sum().reset_index()

sum_variables.head()

,state_county,reported_incidents,substantiated_complaints,infection_control_citations,num_fines,fines_dollar_total,num_payment_denials,total_penalties
0,"AK, Anchorage",34,8,6.0,7,381632.75,0,7
1,"AK, Bethel",4,0,0.0,2,163970.00,0,2
2,"AK, Fairbanks North Star",0,0,1.0,0,0.00,0,0
3,"AK, Juneau",3,1,2.0,3,128495.25,0,3
4,"AK, Kenai Peninsula",18,3,4.0,2,24495.00,0,2


In [33]:
# Next I will gather any variables I'd wish to have as averages at the county level (quality rating scores)
average_variables = nursing_drop.iloc[:,[2,4,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]

# Transformation [7] - groupby.mean()
# Group by county code using the mean values of each group
average_variables['state_county'] = average_variables['state'] + ', '+ average_variables['county']

<ipython-input-33-ddebd6f8fc0d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  average_variables['state_county'] = average_variables['state'] + ', '+ average_variables['county']


In [34]:
average_variables = average_variables.groupby(by = ['state_county']).mean().reset_index()

average_variables.head()

,state_county,overall_rating,health_insp_rating,qm_rating,longstay_qm_rating,shortstay_qm_rating,staffing_rating,rn_staffing_rating,nursing_turnover,rn_turnover,admin_turnover,nurse_aide_hrspres,lpn_hrspres,rn_hrspres,total_nurse_hrspres,total_weighted_health_survey_score
0,"AK, Anchorage",4.000000,3.000000,4.666667,4.000000,5.0,4.666667,4.666667,44.25,41.3,3.333333,3.722833,0.497507,1.992470,6.300777,189.555333
1,"AK, Bethel",3.000000,2.000000,5.000000,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109.333000
2,"AK, Fairbanks North Star",5.000000,3.000000,5.000000,5.000000,5.0,5.000000,5.000000,44.40,34.3,NaN,3.959680,0.911420,2.432280,7.030840,80.667000
3,"AK, Juneau",2.000000,1.000000,5.000000,5.000000,5.0,5.000000,5.000000,52.90,46.2,1.000000,3.858180,0.525030,1.166380,5.521090,237.667000
4,"AK, Kenai Peninsula",4.333333,2.666667,4.333333,4.333333,4.0,5.000000,5.000000,44.05,38.0,0.000000,4.627007,0.601497,2.473913,7.541850,81.444333


In [35]:
# Transformation[8] -  final mergesfor county data
# Merge the avereages variables and sum variables with the county df I have been building
county = county.merge(average_variables, on = ['state_county'], how = 'inner')

In [36]:
# Merge sum variables
county_nursing = county.merge(sum_variables, on = ['state_county'], how = 'inner')

In [37]:
# View final df for county level
county_nursing.head()

,state,county,state_county,nursing_home_count,overall_rating,health_insp_rating,qm_rating,longstay_qm_rating,shortstay_qm_rating,staffing_rating,...,rn_hrspres,total_nurse_hrspres,total_weighted_health_survey_score,reported_incidents,substantiated_complaints,infection_control_citations,num_fines,fines_dollar_total,num_payment_denials,total_penalties
0,AL,Franklin,"AL, Franklin",4,4.250000,4.000000,4.250000,4.500000,3.500000,4.000000,...,1.087187,4.053368,11.500000,0,0,0.0,1,655.14,0,1
1,AL,Talladega,"AL, Talladega",3,4.000000,3.666667,2.666667,2.000000,2.333333,3.666667,...,0.740560,4.362443,13.778000,0,0,0.0,1,650.00,0,1
2,AL,Jackson,"AL, Jackson",3,2.333333,2.666667,2.333333,2.000000,3.000000,2.000000,...,0.776800,4.093410,20.222333,0,0,1.0,3,2932.71,0,3
3,AL,Jefferson,"AL, Jefferson",34,2.147059,2.088235,3.117647,3.147059,3.071429,2.647059,...,0.567236,3.723016,34.666647,9,39,16.0,37,343424.26,1,38
4,AL,Limestone,"AL, Limestone",2,2.000000,2.000000,3.500000,3.500000,3.500000,2.500000,...,0.499470,3.189200,23.000000,3,4,0.0,0,0.00,0,0


In [38]:
# View Rows with Null quality rows to see if other columns may have values > 0 
county_nursing[county_nursing['overall_rating'].isnull()]

,state,county,state_county,nursing_home_count,overall_rating,health_insp_rating,qm_rating,longstay_qm_rating,shortstay_qm_rating,staffing_rating,...,rn_hrspres,total_nurse_hrspres,total_weighted_health_survey_score,reported_incidents,substantiated_complaints,infection_control_citations,num_fines,fines_dollar_total,num_payment_denials,total_penalties
436,GA,Dooly,"GA, Dooly",1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.13204,3.02580,90.000,0,7,2.0,11,117884.25,1,12
458,GA,Clinch,"GA, Clinch",1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.21827,3.00556,52.667,0,4,0.0,8,32055.24,1,9
492,ID,Clearwater,"ID, Clearwater",1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.86793,3.78750,34.000,0,1,1.0,1,3250.00,0,1
880,KS,Logan,"KS, Logan",1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0.0,1,650.00,0,1
1605,NE,Cheyenne,"NE, Cheyenne",1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0.0,0,0.00,0,0
2232,SD,Bennett,"SD, Bennett",1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,5,1.0,2,10400.00,0,2
2333,TN,Bledsoe,"TN, Bledsoe",1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0.0,0,0.00,0,0
2580,VA,Mathews,"VA, Mathews",1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0.0,0,0.00,0,0
2756,CA,Glenn,"CA, Glenn",1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.44048,3.70509,218.333,19,16,2.0,3,65358.25,2,5
2879,TX,Refugio,"TX, Refugio",1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0.0,0,0.00,0,0


In [39]:
# Get state count total
len(county_nursing['state'].value_counts())

53

In [40]:
# Rename nursing home df for easier reference
nursing_homes = nursing_drop

The transformation I have complete in this milestone have left me with 2 useful data frames to use for analyses. One being a large, detailed, data frame of all nursing home care providers in the US ('nursing_homes'). Another being a county level data frame of more targeted variables representing sums or averages based on appropriateness for the variable. With additional time, I will repeat the steps used at the county level at the state level, to create a final, third data frame for insights. To merge with previous dataframes, I will also have to convert column values for state as these are abbreviated, and others are spelled-out. 

In [41]:
# View columns and index
for col in county_nursing.columns:
    index_no = county_nursing.columns.get_loc(col)
    print(index_no , col, )

0 state
1 county
2 state_county
3 nursing_home_count
4 overall_rating
5 health_insp_rating
6 qm_rating
7 longstay_qm_rating
8 shortstay_qm_rating
9 staffing_rating
10 rn_staffing_rating
11 nursing_turnover
12 rn_turnover
13 admin_turnover
14 nurse_aide_hrspres
15 lpn_hrspres
16 rn_hrspres
17 total_nurse_hrspres
18 total_weighted_health_survey_score
19 reported_incidents
20 substantiated_complaints
21 infection_control_citations
22 num_fines
23 fines_dollar_total
24 num_payment_denials
25 total_penalties


In [42]:
# Choose columns to keep and group by state
state1_sum = county_nursing.iloc[:,[0,3,19,20,21,22,23,24,25]].groupby(['state']).sum().reset_index()
len(state1_sum)

53

In [43]:
state1_sum.head()

,state,nursing_home_count,reported_incidents,substantiated_complaints,infection_control_citations,num_fines,fines_dollar_total,num_payment_denials,total_penalties
0,AK,20,80,20,24.0,49,876398.07,0,49
1,AL,227,62,113,60.0,198,2366700.84,7,205
2,AR,220,58,1048,955.0,381,4196949.58,44,425
3,AZ,142,219,151,220.0,235,1803487.13,11,246
4,CA,1176,4470,8543,1683.0,1996,21527811.48,77,2073


In [44]:
state2_mean = county_nursing.iloc[:,[0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]].groupby(['state']).mean().reset_index()
len(state2_mean)

53

In [45]:
state2_mean.head()

,state,overall_rating,health_insp_rating,qm_rating,longstay_qm_rating,shortstay_qm_rating,staffing_rating,rn_staffing_rating,nursing_turnover,rn_turnover,admin_turnover,nurse_aide_hrspres,lpn_hrspres,rn_hrspres,total_nurse_hrspres,total_weighted_health_survey_score
0,AK,3.845238,2.833333,4.000000,3.845238,4.700000,4.820513,4.820513,53.363636,46.960000,0.761905,4.902466,0.675314,2.905411,8.293272,100.276226
1,AL,3.658814,3.186587,3.646851,3.486620,3.556989,3.396424,3.315619,47.544325,38.697449,1.070413,2.450558,1.005773,0.727197,4.135257,22.161920
2,AR,3.282633,3.051536,3.582002,3.798347,3.296962,2.799434,2.325557,55.522657,51.129407,0.566138,2.642978,1.136879,0.502683,4.268722,72.467687
3,AZ,2.994022,2.460344,3.859075,3.836706,3.816017,3.089912,3.404657,55.109597,55.807044,1.001046,2.061405,0.876547,0.834153,3.741256,82.698359
4,CA,3.281232,2.863660,4.123919,4.105979,4.038253,2.780942,2.437500,49.731305,53.019569,1.001708,2.601414,1.234973,0.637654,4.434408,89.420922


In [46]:
state_nursing = state2_mean.merge(state1_sum, on = ['state'], how = "inner")

In [47]:
state_nursing.head()

,state,overall_rating,health_insp_rating,qm_rating,longstay_qm_rating,shortstay_qm_rating,staffing_rating,rn_staffing_rating,nursing_turnover,rn_turnover,...,total_nurse_hrspres,total_weighted_health_survey_score,nursing_home_count,reported_incidents,substantiated_complaints,infection_control_citations,num_fines,fines_dollar_total,num_payment_denials,total_penalties
0,AK,3.845238,2.833333,4.000000,3.845238,4.700000,4.820513,4.820513,53.363636,46.960000,...,8.293272,100.276226,20,80,20,24.0,49,876398.07,0,49
1,AL,3.658814,3.186587,3.646851,3.486620,3.556989,3.396424,3.315619,47.544325,38.697449,...,4.135257,22.161920,227,62,113,60.0,198,2366700.84,7,205
2,AR,3.282633,3.051536,3.582002,3.798347,3.296962,2.799434,2.325557,55.522657,51.129407,...,4.268722,72.467687,220,58,1048,955.0,381,4196949.58,44,425
3,AZ,2.994022,2.460344,3.859075,3.836706,3.816017,3.089912,3.404657,55.109597,55.807044,...,3.741256,82.698359,142,219,151,220.0,235,1803487.13,11,246
4,CA,3.281232,2.863660,4.123919,4.105979,4.038253,2.780942,2.437500,49.731305,53.019569,...,4.434408,89.420922,1176,4470,8543,1683.0,1996,21527811.48,77,2073


In [48]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'}

In [49]:
state_nursing['state'] = state_nursing['state'].replace(states)

In [50]:
county_nursing['state'] = state_nursing['state'].replace(states)

In [51]:
cols = ['overall_rating', 'health_insp_rating', 'qm_rating',
       'longstay_qm_rating', 'shortstay_qm_rating', 'staffing_rating',
       'rn_staffing_rating', 'nursing_turnover', 'rn_turnover',
       'admin_turnover', 'nurse_aide_hrspres', 'lpn_hrspres', 'rn_hrspres',
       'total_nurse_hrspres']

state_nursing[cols] = state_nursing[cols].round(2)
county_nursing[cols] = county_nursing[cols].round(2)


In [52]:
state_nursing.head()

,state,overall_rating,health_insp_rating,qm_rating,longstay_qm_rating,shortstay_qm_rating,staffing_rating,rn_staffing_rating,nursing_turnover,rn_turnover,...,total_nurse_hrspres,total_weighted_health_survey_score,nursing_home_count,reported_incidents,substantiated_complaints,infection_control_citations,num_fines,fines_dollar_total,num_payment_denials,total_penalties
0,Alaska,3.85,2.83,4.00,3.85,4.70,4.82,4.82,53.36,46.96,...,8.29,100.276226,20,80,20,24.0,49,876398.07,0,49
1,Alabama,3.66,3.19,3.65,3.49,3.56,3.40,3.32,47.54,38.70,...,4.14,22.161920,227,62,113,60.0,198,2366700.84,7,205
2,Arkansas,3.28,3.05,3.58,3.80,3.30,2.80,2.33,55.52,51.13,...,4.27,72.467687,220,58,1048,955.0,381,4196949.58,44,425
3,Arizona,2.99,2.46,3.86,3.84,3.82,3.09,3.40,55.11,55.81,...,3.74,82.698359,142,219,151,220.0,235,1803487.13,11,246
4,California,3.28,2.86,4.12,4.11,4.04,2.78,2.44,49.73,53.02,...,4.43,89.420922,1176,4470,8543,1683.0,1996,21527811.48,77,2073


In [53]:
county_nursing = county_nursing.drop(columns = ['state', 'county'])
county_nursing.head()

,state_county,nursing_home_count,overall_rating,health_insp_rating,qm_rating,longstay_qm_rating,shortstay_qm_rating,staffing_rating,rn_staffing_rating,nursing_turnover,...,rn_hrspres,total_nurse_hrspres,total_weighted_health_survey_score,reported_incidents,substantiated_complaints,infection_control_citations,num_fines,fines_dollar_total,num_payment_denials,total_penalties
0,"AL, Franklin",4,4.25,4.00,4.25,4.50,3.50,4.00,4.50,54.10,...,1.09,4.05,11.500000,0,0,0.0,1,655.14,0,1
1,"AL, Talladega",3,4.00,3.67,2.67,2.00,2.33,3.67,3.33,56.30,...,0.74,4.36,13.778000,0,0,0.0,1,650.00,0,1
2,"AL, Jackson",3,2.33,2.67,2.33,2.00,3.00,2.00,2.00,52.50,...,0.78,4.09,20.222333,0,0,1.0,3,2932.71,0,3
3,"AL, Jefferson",34,2.15,2.09,3.12,3.15,3.07,2.65,2.59,58.28,...,0.57,3.72,34.666647,9,39,16.0,37,343424.26,1,38
4,"AL, Limestone",2,2.00,2.00,3.50,3.50,3.50,2.50,2.50,50.00,...,0.50,3.19,23.000000,3,4,0.0,0,0.00,0,0


In [54]:
# SAVE CSV
county_nursing.to_csv('county_nursing.csv', index = False)
state_nursing.to_csv('state_nursing.csv', index = False)